```{contents}
```
## **Join in LangGraph**

In LangGraph, a **Join** is a control-flow mechanism that **synchronizes multiple parallel execution paths** and merges their outputs into a single downstream path.
It is fundamental for building **parallel, multi-agent, and distributed LLM workflows**.

---

### **1. Motivation: Why Join Is Needed**

Modern LLM systems often execute tasks **in parallel**:

* Run multiple tools concurrently
* Ask multiple agents to solve the same problem
* Split a task into independent subtasks

However, **a final decision requires all partial results**.
The **Join** node enforces this synchronization.

```
        Task A ──┐
                 ├──► JOIN ───► Final Node
        Task B ──┘
```

---

### **2. Conceptual Model**

A Join implements the classical **barrier synchronization** pattern.

| Property      | Description                       |
| ------------- | --------------------------------- |
| Blocking      | Waits for all upstream branches   |
| Deterministic | Guarantees all inputs are present |
| State-driven  | Merges via reducers               |
| Fault-aware   | Can handle partial failures       |

---

### **3. How Join Works in LangGraph**

LangGraph has no explicit "Join" keyword.
A Join emerges from **state reducers + multiple incoming edges**.

#### **State Schema with Reducer**

```python
from typing import TypedDict, List, Annotated
from langgraph.graph import add_messages

class State(TypedDict):
    results: Annotated[List[str], add_messages]
```

`add_messages` is a reducer that **accumulates** results from all branches.

---

### **4. Minimal Join Example**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Annotated
from langgraph.graph import add_messages

class State(TypedDict):
    results: Annotated[List[str], add_messages]

def agent_a(state):
    return {"results": ["A done"]}

def agent_b(state):
    return {"results": ["B done"]}

def final_node(state):
    return {"results": ["Final: " + " & ".join(state["results"])]}

builder = StateGraph(State)

builder.add_node("A", agent_a)
builder.add_node("B", agent_b)
builder.add_node("final", final_node)

builder.set_entry_point("A")
builder.add_edge("A", "final")
builder.add_edge("B", "final")

builder.set_entry_point("B")

graph = builder.compile()
print(graph.invoke({"results": []}))
```

**Execution**

```
A and B run in parallel
→ Join at "final"
→ results = ["A done", "B done"]
```

---

### **5. Join in Multi-Agent Systems**

```
Planner ─┬─ Research Agent ─┐
         ├─ Compute Agent ──┼─► JOIN ─► Decision
         └─ Verify Agent ───┘
```

The Join ensures:

* All agents complete
* Their outputs are merged
* Final decision uses full context

---

### **6. Variants of Join**

| Variant           | Description                 |
| ----------------- | --------------------------- |
| Hard Join         | Waits for all branches      |
| Soft Join         | Proceeds after quorum       |
| Timed Join        | Proceeds after timeout      |
| Priority Join     | Some branches weighted more |
| Conditional Join  | Merge depends on state      |
| Hierarchical Join | Join of joins (nested)      |

---

### **7. Production Considerations**

| Concern         | Handling           |
| --------------- | ------------------ |
| Deadlock        | Timeout + fallback |
| Partial failure | Retry policies     |
| Large merges    | Streaming reducers |
| Latency         | Parallelism tuning |
| Cost            | Budget-aware joins |

---

### **8. When to Use Join**

Use Join whenever:

* Parallel work is split
* Multiple agents collaborate
* Tools execute concurrently
* Consensus is required
* Results must be reconciled

---

### **9. Mental Model**

> **Fan-out → Parallel Work → Join → Unified Decision**

Join is what transforms LangGraph from a **pipeline** into a **true distributed reasoning system**.


### Demonstration

In [2]:
from typing import TypedDict, List, Annotated
from langgraph.graph import StateGraph, END

# ---- Custom reducer for string merging ----
def merge_results(a: List[str], b: List[str]) -> List[str]:
    return a + b

# ---- 1. Define State ----
class State(TypedDict):
    results: Annotated[List[str], merge_results]

# ---- 2. Parallel Workers ----
def agent_a(state):
    return {"results": ["Agent A finished"]}

def agent_b(state):
    return {"results": ["Agent B finished"]}

# ---- 3. Join Node ----
def join_node(state):
    merged = " | ".join(state["results"])
    return {"results": [f"JOIN RESULT: {merged}"]}

# ---- 4. Build Graph ----
builder = StateGraph(State)

builder.add_node("A", agent_a)
builder.add_node("B", agent_b)
builder.add_node("JOIN", join_node)

# Fan-out + Join
builder.set_entry_point("A")
builder.add_edge("A", "JOIN")
builder.add_edge("B", "JOIN")
builder.set_entry_point("B")

graph = builder.compile()

# ---- 5. Run ----
output = graph.invoke({"results": []})
print(output)


{'results': ['Agent A finished', 'Agent B finished', 'JOIN RESULT: Agent A finished | Agent B finished']}
